In [17]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/srv/main-spack-instance/spack/var/spack/environments/release_v0_1_202109_hotfix/.spack-env/view/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['hamming']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [18]:
from ray import tune
import numpy as np
import pandas as pd
import os
import datetime
from ray.tune.suggest.bayesopt import BayesOptSearch
from one_region_simu import *
from evaluation import *
import pdb
import sys

In [19]:
data_folder = os.path.join(os.getcwd(), 'data')
exp_name = '20220623_trial1'
ray_folder = os.path.join(data_folder,'ray_results')
exp_folder = os.path.join(ray_folder, exp_name)
restore_folder = exp_folder
os.makedirs(exp_folder, exist_ok=True)
# os.environ['RAY_PICKLE_VERBOSE_DEBUG']='1'

In [20]:
config={
    'nsig':tune.uniform(0.0001,0.004),
    'sigma': tune.uniform(0.2, 0.5),
        "c_ee": tune.uniform(8, 18),
        'c_ie': tune.uniform(7.5, 10)}

def run_model(config):
    params = config_params(nsig=config['nsig'],sigma=config['sigma'], c_ee=config['c_ee'],c_ie=config['c_ie'])
    # score = eva_limit_cycle(params)
    # if not score:
    region = config_one_region()
    surface = config_surface(region)
    sim = config_simulator(params, region, surface)
    result_name = run_simulation(sim, params, check_point=12500,data_folder=data_folder)
    score, dfa_all, f_peak = dfa_analysis(result_name)
    tune.report(score=score, dfa_all = dfa_all, f_peak=f_peak)

In [21]:
# # Customize the logger
from typing import Dict, List
import json
from ray.tune.logger import LoggerCallback
from tensorboardX import SummaryWriter
from shutil import rmtree

class CustomLoggerCallback(LoggerCallback):
    """Put the results of all trails in the same file"""

    def __init__(self, filefolder, filename: str = "log.txt", bayesopt=None):
        self._filename = filename
        self._filefolder = filefolder
        self._filepath = os.path.join(filefolder, self._filename)
        self.bayesopt = bayesopt
        self.waive_params = ["time_this_iter_s","timesteps_total","episodes_total","timestamp", "pid",
                 "hostname", "node_ip", "timesteps_since_restore", "iterations_since_restore", "warmup_time"]
        
    def log_trial_start(self, trial: "Trial"):
        self._file = open(self._filepath, "at")
        self.summarywriter = SummaryWriter(os.path.join(self._filefolder,'tensor_events'))

    def log_trial_result(self, iteration: int, trial: "Trial", result: Dict):
        self._file = open(self._filepath, "at")
        for k, v in list(result.items()):
            if k in self.waive_params:
                del result[k]
                continue
            if isinstance(v, (np.ndarray,)):
                result[k] = v.tolist()
        self._file.write(json.dumps(result))
        self.summarywriter.add_scalar('score', result['score'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_delta', result['dfa_all']['delta'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_theta', result['dfa_all']['theta'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_alpha', result['dfa_all']['alpha'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_beta', result['dfa_all']['beta'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_gamma', result['dfa_all']['gamma'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_raw', result['dfa_all']['raw'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('dfa_raw', result['dfa_all']['dfa_this_trial'], len(self.bayesopt._buffered_trial_results)+1)
        self.summarywriter.add_scalar('f_peak', result['f_peak'], len(self.bayesopt._buffered_trial_results)+1)

    def on_trial_complete(self, iteration: int, trials: List["Trial"],
                          trial: "Trial", **info):
        self._file = open(self._filepath, "at")
        self._file.write(str(datetime.datetime.now())+'\n')
        self._file.close()

        self.summarywriter.close()

In [ ]:
smoke_test = False
restore = True
nsample = -1
debug = False
bayesopt = BayesOptSearch(metric="score",
                          mode="min",
                          points_to_evaluate=[{"nsig":0.0001,"sigma": 0.3,"c_ee": 8, 'c_ie':8.5}],
                          random_search_steps=150)
# pdb.set_trace()
# if restore:
#     bayesopt.restore(os.path.join(restore_folder, 'baye_checkpoint.pkl'))
if debug:
    sys.path.append('/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune')
    pdb.set_trace()
try:
    analysis = tune.run(run_model,
                        verbose=3,
                        name = exp_name,
                        local_dir=ray_folder,
                        search_alg=bayesopt,
                        # config=config,
                        metric="score",
                        mode="min",
                        progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
                                                                      parameter_columns=config),
                        callbacks=[CustomLoggerCallback(bayesopt=bayesopt, filefolder=exp_folder, filename= "log_test.txt")],
                        stop={'score':0.06},
                        # resume= 'ERRORED_ONLY', # do this when you whant to resume a stopped experiment option:'ERRORED_ONLY'
                        resume= True,
                        # max_concurrent_trials = 3,
                        num_samples=2 if smoke_test else nsample)
except ValueError as e:
    # TODO: Add the error!
    if str(e) == "blablabla":
        analysis = tune.run(run_model,
                            verbose=3,
                            name = exp_name,
                            local_dir=ray_folder,
                            search_alg=bayesopt,
                            # config=config,
                            metric="score",
                            mode="min",
                            progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
                                                                      parameter_columns=config),
                            callbacks=[CustomLoggerCallback(filefolder=exp_folder, filename= "log_test.txt",bayesopt=bayesopt)],
                            stop={'score':0.08},
                            # resume=True, # do this when you whant to resume a stopped experiment
                            max_concurrent_trials = 3,
                            num_samples=2 if smoke_test else nsample)
    elif str(e) ==  "'NoneType' object is not callable":
        pdb.post_mortem()
    else:
        raise
except:
    pdb.post_mortem(sys.exc_info()[2])

Trial name,status,loc,<ray.tune.sample.Float object at 0x7f6771964d30>,<ray.tune.sample.Float object at 0x7f6771964460>,<ray.tune.sample.Float object at 0x7f6771964be0>,<ray.tune.sample.Float object at 0x7f6771964610>
run_model_9f900c7e,RUNNING,10.128.3.134:24126,0.0005664175591593767,0.413973,16.8721,8.68054
run_model_7a0ea71c,PENDING,,0.003106772001822788,0.348139,15.6079,8.90319
run_model_15c584ce,TERMINATED,10.128.3.134:12909,0.0038813800482817786,0.43254,13.4671,7.96214
run_model_182b4b2c,TERMINATED,10.128.3.134:11792,0.001315673196748703,0.35602,10.5878,9.15631
run_model_1ce50d24,TERMINATED,10.128.3.134:14031,0.002431809917363232,0.476562,17.395,9.73707
run_model_2bc29b86,TERMINATED,10.128.3.134:15165,0.0002763864267510985,0.297599,8.88493,7.98996
run_model_4fdcbc86,TERMINATED,10.128.3.134:16285,0.0033320762856925245,0.307026,11.8868,8.17837
run_model_56a3e5f8,TERMINATED,10.128.3.134:19648,0.0028567436410057065,0.418702,8.05522,9.53865
run_model_5964aba6,TERMINATED,10.128.3.134:17406,0.0006496044774015744,0.440659,10.8093,8.85674
run_model_5fefd70c,TERMINATED,10.128.3.134:18517,0.003111754600256964,0.259615,8.74551,9.96722


In [10]:
len(bayesopt._buffered_trial_results)

0

In [11]:
# save the current experiment
save_ckp=True
if save_ckp:
    bayesopt.save(os.path.join(exp_folder, 'baye_checkpoint.pkl'))

In [12]:
print("Best hyperparameters found were: ", analysis.best_config)
print("Best result: ", analysis.best_result)
analysis.results_df

2022-06-23 19:04:21,490	WARNING experiment_analysis.py:533 -- Could not find best trial. Did you pass the correct `metric` parameter?
2022-06-23 19:04:21,492	WARNING experiment_analysis.py:533 -- Could not find best trial. Did you pass the correct `metric` parameter?


Best hyperparameters found were:  None


AttributeError: 'NoneType' object has no attribute 'last_result'

## Load the result from file

In [ ]:
from ray.tune import ExperimentAnalysis
analysis = ExperimentAnalysis(exp_folder)

In [ ]:
analysis.get_best_config(metric="score",mode="min",)

In [ ]:
analysis.get_best_trial(metric="score",mode="min",).last_result

In [ ]:
analysis.trials[0].metric_analysis

In [ ]:
min(analysis.results_df.score)